# 파이토치

- 마지막 AvgPool에서, 논문에 정확한 수치는 없지만 1x1x1536이 되기 위한 숫자를 찾음 (주석 처리한 self.avgpool)


- 근데 그냥 AdaptiveAvgPool2d로 (1,1) 크기로 바로 만들 수 있음, 지정한 avgpool로 하나 globalavgpool로 하나 모델 파라미터 수는 똑같음


- AdaptiveAvgPool2d의 변수 (H,W)로 아웃풋 크기를 지정할 수 있음. 어떤 인풋이 들어와도 저 사이즈대로 맞춰서 만들어줌.

In [1]:
import torch
import torch.nn as nn


class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, stride, padding):
        super(ConvBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding, bias=False)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        x = self.conv(x)
        x = self.relu(x)
        
        return x
    

class Stem(nn.Module):
    def __init__(self):
        super(Stem, self).__init__()
        
        self.branch1 = nn.Sequential(
            ConvBlock(3, 32, kernel_size=3, stride=2, padding=0),
            ConvBlock(32, 32, kernel_size=3, stride=1, padding=0),
            ConvBlock(32, 64, kernel_size=3, stride=1, padding=1))
        
        self.maxpool_96 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        self.maxpool_192 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.conv_96 = ConvBlock(64, 96, kernel_size=3, stride=2, padding=0)
        self.conv_192 = ConvBlock(192, 192, kernel_size=3, stride=2, padding=0) # 논문엔 stride = 2 표시가 안된듯
        
        self.branch2_1 = nn.Sequential(
            ConvBlock(160, 64, kernel_size=1, stride=1, padding=0),
            ConvBlock(64, 96, kernel_size=3, stride=1, padding=0))
        
        self.branch2_2 = nn.Sequential(
            ConvBlock(160, 64, kernel_size=1, stride=1, padding=0),
            ConvBlock(64, 64, kernel_size=(7,1), stride=1, padding=(3,0)),
            ConvBlock(64, 64, kernel_size=(1,7), stride=1, padding=(0,3)),
            ConvBlock(64, 96, kernel_size=3, stride=1, padding=0))
        
    def forward(self, x):
        
        x = self.branch1(x)
        
        x1_1 = self.maxpool_96(x)
        x1_2 = self.conv_96(x)
        
        x = torch.cat([x1_1, x1_2], dim=1)
        
        x2_1 = self.branch2_1(x)
        x2_2 = self.branch2_2(x)
        
        x = torch.cat([x2_1, x2_2], dim=1)
        
        x3_1 = self.conv_192(x)
        x3_2 = self.maxpool_192(x)
        
        x = torch.cat([x3_1, x3_2], dim=1)
        
        return x
    
    
class InceptionA(nn.Module):
    def __init__(self, in_ch):
        super(InceptionA, self).__init__()
        
        self.branch1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),          # AvgPool 정보가 논문엔 없는데 이렇게 하니까 됨
            ConvBlock(in_ch, 96, kernel_size=1, stride=1, padding=0))
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, 96, kernel_size=1, stride=1, padding=0))
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_ch, 64, kernel_size=1, stride=1, padding=0),
            ConvBlock(64, 96, kernel_size=3, stride=1, padding=1))
        
        self.branch4 = nn.Sequential(
            ConvBlock(in_ch, 64, kernel_size=1, stride=1, padding=0),
            ConvBlock(64, 96, kernel_size=3, stride=1, padding=1),
            ConvBlock(96, 96, kernel_size=3, stride=1, padding=1))
        
    def forward(self, x):
        
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        
        return torch.cat([x1, x2, x3, x4], dim=1)
    
    
class ReductionA(nn.Module): # k=192, l=224, m=256, n=384
    def __init__(self, in_ch, k, l, m, n):
        super(ReductionA, self).__init__()
        
        self.branch1 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0))
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, n, kernel_size=3, stride=2, padding=0))
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_ch, k, kernel_size=1, stride=1, padding=0),
            ConvBlock(k, l, kernel_size=3, stride=1, padding=1),
            ConvBlock(l, m, kernel_size=3, stride=2, padding=0))
        
    def forward(self, x):
        
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        
        return torch.cat([x1, x2, x3], dim=1)
    
    
class InceptionB(nn.Module):
    def __init__(self, in_ch):
        super(InceptionB, self).__init__()
        
        self.branch1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_ch, 128, kernel_size=1, stride=1, padding=0))
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, 384, kernel_size=1, stride=1, padding=0))
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_ch, 192, kernel_size=1, stride=1, padding=0),
            ConvBlock(192, 224, kernel_size=(1,7), stride=1, padding=(0,3)),
            ConvBlock(224, 256, kernel_size=(1,7), stride=1, padding=(0,3)))
        
        self.branch4 = nn.Sequential(
            ConvBlock(in_ch, 192, kernel_size=1, stride=1, padding=0),
            ConvBlock(192, 192, kernel_size=(1,7), stride=1, padding=(0,3)),
            ConvBlock(192, 224, kernel_size=(7,1), stride=1, padding=(3,0)),
            ConvBlock(224, 224, kernel_size=(1,7), stride=1, padding=(0,3)),
            ConvBlock(224, 256, kernel_size=(7,1), stride=1, padding=(3,0)))
        
    def forward(self, x):
        
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        
        return torch.cat([x1, x2, x3, x4], dim=1)
    
    
class ReductionB(nn.Module):
    def __init__(self, in_ch):
        super(ReductionB, self).__init__()
        
        self.branch1 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0))
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, 192, kernel_size=1, stride=1, padding=0),
            ConvBlock(192, 192, kernel_size=3, stride=2, padding=0))
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_ch, 256, kernel_size=1, stride=1, padding=0),
            ConvBlock(256, 256, kernel_size=(1,7), stride=1, padding=(0,3)),
            ConvBlock(256, 320, kernel_size=(7,1), stride=1, padding=(3,0)),
            ConvBlock(320, 320, kernel_size=3, stride=2, padding=0))
        
    def forward(self, x):
        
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        
        return torch.cat([x1, x2, x3], dim=1)
    
    
class InceptionC(nn.Module):
    def __init__(self, in_ch):
        super(InceptionC, self).__init__()
        
        self.branch1 = nn.Sequential(
            nn.AvgPool2d(kernel_size=3, stride=1, padding=1),
            ConvBlock(in_ch, 256, kernel_size=1, stride=1, padding=0))
        
        self.branch2 = nn.Sequential(
            ConvBlock(in_ch, 256, kernel_size=1, stride=1, padding=0))
        
        self.branch3 = nn.Sequential(
            ConvBlock(in_ch, 384, kernel_size=1, stride=1, padding=0))
        
        self.conv1x3_br3 = ConvBlock(384, 256, kernel_size=(3,1), stride=1, padding=(1,0))
        self.conv3x1_br3 = ConvBlock(384, 256, kernel_size=(1,3), stride=1, padding=(0,1))
        
        self.branch4 = nn.Sequential(
            ConvBlock(in_ch, 384, kernel_size=1, stride=1, padding=0),
            ConvBlock(384, 448, kernel_size=(1,3), stride=1, padding=(0,1)),
            ConvBlock(448, 512, kernel_size=(3,1), stride=1, padding=(1,0)))
        
        self.conv1x3_br4 = ConvBlock(512, 256, kernel_size=(3,1), stride=1, padding=(1,0))
        self.conv3x1_br4 = ConvBlock(512, 256, kernel_size=(1,3), stride=1, padding=(0,1))
        
    def forward(self, x):
        
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        
        x3 = self.branch3(x)
        
        x3_1 = self.conv1x3_br3(x3)
        x3_2 = self.conv3x1_br3(x3)
        
        x3 = torch.cat([x3_1, x3_2], dim=1)
        
        x4 = self.branch4(x)
        
        x4_1 = self.conv1x3_br4(x4)
        x4_2 = self.conv3x1_br4(x4)
        
        x4 = torch.cat([x4_1, x4_2], dim=1)
        
        return torch.cat([x1, x2, x3, x4], dim=1)

In [2]:
class InceptionV4(nn.Module):
    def __init__(self, num_classes = 1000):
        super(InceptionV4, self).__init__()
        
        self.stem = Stem()
        
        self.inception_a = InceptionA(384)
        self.inception_b = InceptionB(1024)
        self.inception_c = InceptionC(1536)
        
        self.red_a = ReductionA(384, 192, 224, 256, 384)
        self.red_b = ReductionB(1024)
        
        # self.avgpool = nn.AvgPool2d(kernel_size=7, stride=2, padding=0)
        
        self.globalavgpool = nn.AdaptiveAvgPool2d((1,1))
        
        self.dropout = nn.Dropout(0.8)
        
        self.linear = nn.Linear(1536, num_classes)
        
    def forward(self, x):
        
        x = self.stem(x)
        
        for _ in range(4):
            x = self.inception_a(x)
            
        x = self.red_a(x)
        
        for _ in range(7):
            x = self.inception_b(x)
            
        x = self.red_b(x)
        
        for _ in range(3):
            x = self.inception_c(x)
            
        x = self.globalavgpool(x)
        x = self.dropout(x)
        x = x.view(x.size(0),-1)
        x = self.linear(x)
            
        return x

In [3]:
if __name__ == '__main__':

    from torchsummary import summary
    model = InceptionV4()
    summary(model, (3,299,299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
              ReLU-2         [-1, 32, 149, 149]               0
         ConvBlock-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 32, 147, 147]           9,216
              ReLU-5         [-1, 32, 147, 147]               0
         ConvBlock-6         [-1, 32, 147, 147]               0
            Conv2d-7         [-1, 64, 147, 147]          18,432
              ReLU-8         [-1, 64, 147, 147]               0
         ConvBlock-9         [-1, 64, 147, 147]               0
        MaxPool2d-10           [-1, 64, 73, 73]               0
           Conv2d-11           [-1, 96, 73, 73]          55,296
             ReLU-12           [-1, 96, 73, 73]               0
        ConvBlock-13           [-1, 96, 73, 73]               0
           Conv2d-14           [-1, 64,